# Homework w04d01

## Web Scrapping.

- For this homework you need to use Beautiful soup to scrape a website and store the results in a csv.

    1 - Use BeautifulSoup to get and parse data from a website that includes a table with information. Hint: For more information about how HTML defines tables, read: http://www.w3schools.com/html/html_tables.asp
    
    2 - Store the table in a pandas data frame, and print the first couple of rows.
    
    3 - Perform some transformation on the data. For ex: adding a column that aggregates information, or changing True/False values to Booleans (0/1). If there's a numerical column you can add a "total amount" row.
    
    4 - Print the first couple of values of the transformed dataframe again.
    
    5 - Store the new dataframe as a csv.

In [171]:
# Get webpage.
from bs4 import BeautifulSoup
import urllib
r = urllib.urlopen('http://www.ikea.com/fr/fr/search/?query=bureau').read()

In [173]:
# Load table into dataframe.
soup = BeautifulSoup(r)
print type(soup)
print soup.prettify()[0:500]

<class 'bs4.BeautifulSoup'>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title id="htmlTitle">
   Résultat de la recherche - IKEA
  </title>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="2016-10-31" name="date_published" sche


In [174]:
# Identify tables containing product information
desks = soup.find_all("div", class_="productPadding")
print desks[0].prettify()

<div class="productPadding eS-click eS-t-Oy9zZWFyY2gtYWxsL3Byb2R1Y3QtcmVzdWx0L3NlYXJjaC1oaXRzLXdpdGgtY291bnQvc2VhcmNoLWhpdHM7Iztwcm9kdWN0X2tleTsyNzk2OV9mcl9GUjswMDI2Mzg1MF9mcl9GUjs7Izs4Njs">
 <a href="/fr/fr/catalog/products/00263850/" onclick="irwStatTopProductClicked();">
  <img alt="BILLY Bibliothèque Largeur: 80 cm Profondeur: 28 cm Hauteur: 202 cm" border="0" class="prodImg" id="imgThmbProduct1" src="/PIAimages/0252367_PE391149_S2.JPG"/>
  <span class="prodName prodNameTro" id="txtNameProduct1">
   BILLY
  </span>
  <div>
   <span class="prodDesc" id="txtDescrProduct1" style="display:inline;">
    Bibliothèque
   </span>
  </div>
  <span class="prodPrice" id="txtPriceProduct1" style="clear:both;">
   39,90 €
  </span>
  <div id="comparisonContainer" style="display: none">
   <span>
    Prix unitaire
   </span>
   <span class="packagepricevaluebold" id="comparisonPrice">
   </span>
  </div>
 </a>
</div>



In [178]:
names = soup.find_all("span", class_="prodName")
prices = soup.find_all("span", class_="prodPrice")
import pandas as pd

# Extract item name and price to a dataframe
res = pd.DataFrame([(names[i].contents[0],prices[i].contents[0]) for i in range(len(names))])

In [179]:
df = pd.DataFrame(res)
df

,0,1
0,BILLY,"39,90 €"
1,VOLMAR,199 €
2,MARKUS,185 €
3,KALLAX,65 €
4,RENBERGET,"39,90 €"
5,KALLAX,169 €
6,BILLY,29 €
7,KALLAX,99 €
8,ERIK,59 €
9,FLINTAN,75 €


In [180]:
# Display price item
df[1][0]

u'\r\n\t\t\t\t39,90 \u20ac\t\t\t\t\r\n\t\t\t'

In [181]:
# Convert price to a float
import re
df[1] = df[1].apply(lambda x: (re.findall(r'[0-9]+,{0,1}[0-9]{0,2}',x))[0])
df[1] = df[1].apply(lambda x: float(re.sub(r',',r'.',x)))

In [182]:
df

,0,1
0,BILLY,39.90
1,VOLMAR,199.00
2,MARKUS,185.00
3,KALLAX,65.00
4,RENBERGET,39.90
5,KALLAX,169.00
6,BILLY,29.00
7,KALLAX,99.00
8,ERIK,59.00
9,FLINTAN,75.00


In [184]:
# Add delta column comparing item price with mean price for similar items
import numpy as np
mean = np.mean(df[1])
df["Delta"] = [n - mean for n in df[1]]
df

,0,1,Delta
0,BILLY,39.90,-62.682
1,VOLMAR,199.00,96.418
2,MARKUS,185.00,82.418
3,KALLAX,65.00,-37.582
4,RENBERGET,39.90,-62.682
5,KALLAX,169.00,66.418
6,BILLY,29.00,-73.582
7,KALLAX,99.00,-3.582
8,ERIK,59.00,-43.582
9,FLINTAN,75.00,-27.582


In [186]:
df.to_csv("./desks_ikea.csv",encoding="utf-8")